In [3]:
from mlrun.config import config
from mlrun import code_to_function, auto_mount
import requests

### Update Model Path
The `/User` directory is automatically mapped to your home directory. If you are the `admin` user, this will be equivalent to `/v3io/users/admin`. Because of how Nuclio is reading from the file-system, we need to use this format. You can run `pwd` in your notebook to get the required path (assuming your model is in the same directory as your notebook).

In [4]:
!pwd

/User/igz_repos/igz-royal-cyber/spark_serving


In [5]:
MODEL_PATH = "file:///User/igz_repos/igz-royal-cyber/spark_serving/sampleModel"

### Configure Nuclio Function

In [6]:
def get_spark_image():
    app_image = config.spark_app_image.replace("spark-app", "shell")
    image_tag = config.spark_app_image_tag
    return f"{app_image}:{image_tag}"

In [7]:
serving_fn = code_to_function(
    name='spark-serving',
    project="spark-pipeline",
    filename="spark_serving.py",
    kind='nuclio',
    handler="handler",
    requirements=["pyspark", "numpy", "pandas"],
    image=get_spark_image(),
).apply(auto_mount())

# Disable auto-scaling
serving_fn.spec.min_replicas = 1
serving_fn.spec.max_replicas = 1

# Specify CPU resources
serving_fn.with_requests(cpu=1)
serving_fn.with_limits(cpu=1)

# Set model path as ENV var
serving_fn.set_env("MODEL_PATH", MODEL_PATH)

### Deploy Nuclio Function

In [8]:
url = serving_fn.deploy()

> 2021-12-15 19:52:12,383 [info] Starting remote function deploy
2021-12-15 19:52:12  (info) Deploying function
2021-12-15 19:52:12  (info) Building
2021-12-15 19:52:12  (info) Staging files and preparing base images
2021-12-15 19:52:12  (info) Building processor image
2021-12-15 19:52:14  (info) Build complete
2021-12-15 19:52:39  (info) Function deploy complete
> 2021-12-15 19:52:40,352 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-spark-pipeline-spark-serving.default-tenant.svc.cluster.local:8080'], 'external_invocation_urls': ['spark-pipeline-spark-serving-spark-pipeline.default-tenant.app.us-sales-32.iguazio-cd0.com/']}


### Invoke Nuclio Function

In [9]:
data = {'param1': -133144.09, 'param2': 1.0, 'param3': 0.0}

##### Via MLRun Function Object

In [11]:
%%time
serving_fn.invoke("/", body=data)

> 2021-12-15 19:52:48,031 [info] invoking function: {'method': 'POST', 'path': 'http://nuclio-spark-pipeline-spark-serving.default-tenant.svc.cluster.local:8080/'}
CPU times: user 12.8 ms, sys: 0 ns, total: 12.8 ms
Wall time: 397 ms


b'{"features":{"type":1,"values":[-133144.09,1.0,0.0]},"rawPrediction":{"type":1,"values":[19.993485318561074,0.00651468143892579]},"probability":{"type":1,"values":[0.9996742659280538,3.257340719462895E-4]},"prediction":0.0}'

##### Via Post Request

In [12]:
print(url)

http://spark-pipeline-spark-serving-spark-pipeline.default-tenant.app.us-sales-32.iguazio-cd0.com/


In [13]:
%%time
resp = requests.post(url=url, json=data)
resp.json()

CPU times: user 11.7 ms, sys: 0 ns, total: 11.7 ms
Wall time: 468 ms


{'features': {'type': 1, 'values': [-133144.09, 1.0, 0.0]},
 'rawPrediction': {'type': 1,
  'values': [19.993485318561074, 0.00651468143892579]},
 'probability': {'type': 1,
  'values': [0.9996742659280538, 0.0003257340719462895]},
 'prediction': 0.0}